<a href="https://colab.research.google.com/github/kumastry/cs_stress_majorization/blob/main/%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E7%A7%91%E5%AD%A6%E7%89%B9%E8%AB%961.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -o graph.zip https://esa-storage-tokyo.s3-ap-northeast-1.amazonaws.com/uploads/production/attachments/8704/2022/06/24/28750/8be3d27d-7c30-471a-a609-ce2ed4ba50fc.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  214k  100  214k    0     0   149k      0  0:00:01  0:00:01 --:--:--  149k


In [ ]:
!mkdir -p result

In [ ]:
!unzip -o graph.zip

Archive:  graph.zip
  inflating: graph/1138_bus.json     
  inflating: graph/3elt.json         
  inflating: graph/bull.json         
  inflating: graph/chvatal.json      
  inflating: graph/cubical.json      
  inflating: graph/davis_southern_women.json  
  inflating: graph/desargues.json    
  inflating: graph/diamond.json      
  inflating: graph/dodecahedral.json  
  inflating: graph/dwt_1005.json     
  inflating: graph/dwt_2680.json     
  inflating: graph/florentine_families.json  
  inflating: graph/frucht.json       
  inflating: graph/heawood.json      
  inflating: graph/hoffman_singleton.json  
  inflating: graph/house.json        
  inflating: graph/house_x.json      
  inflating: graph/icosahedral.json  
  inflating: graph/karate_club.json  
  inflating: graph/krackhardt_kite.json  
  inflating: graph/les_miserables.json  
  inflating: graph/moebius_kantor.json  
  inflating: graph/octahedral.json   
  inflating: graph/pappus.json       
  inflating: graph/petersen.json  

In [ ]:
UNIT_EDGE_LENGTH = 30

In [ ]:
import glob
import json
import os.path
import pandas as pd
import networkx as nx
records = []
for filename in glob.glob('graph/*.json'):
    graph = nx.node_link_graph(json.load(open(filename)))
    records.append([
        os.path.basename(filename),
        graph.number_of_nodes(),
        graph.number_of_edges(),
        nx.is_connected(graph),
    ])
df = pd.DataFrame(records, columns=['filename', 'number_of_nodes', 'number_of_edges', 'is_connected'])
df

,filename,number_of_nodes,number_of_edges,is_connected
0,les_miserables.json,77,254,True
1,hoffman_singleton.json,50,175,True
2,dodecahedral.json,20,30,True
3,florentine_families.json,15,20,True
4,krackhardt_kite.json,10,18,True
5,tutte.json,46,69,True
6,petersen.json,10,15,True
7,cubical.json,8,12,True
8,frucht.json,12,18,True
9,1138_bus.json,1138,1458,True


In [ ]:
from math import sqrt
from random import random
from networkx import floyd_warshall_numpy
from scipy.sparse.linalg import cg
import numpy as np

def inv(x):
    if x < 1e-4:
        return 0.0
    return 1 / x


def stress(X, D):
    n = len(X)
    s = 0
    for i in range(n):
        X_i = X[i, :]
        for j in range(i):
            X_j = X[j, :]
            d = np.linalg.norm(X_i - X_j) - D[i, j]
            s += d * d
    return s



def stress_majorization(graph):
    epsilon = 1e-4
    n = graph.number_of_nodes()
    D = floyd_warshall_numpy(graph)
    print(D)
    w = np.zeros((n, n))
    delta = np.zeros((n, n))
    for i in range(n):
        for j in range(i):
            w[i, j] = w[j, i] = D[i, j] ** -2
            delta[i, j] = delta[j, i] = w[i, j] * D[i, j]
    Z = np.random.rand(n, 2)
    Z[0] = (0., 0.)
    L_w = np.zeros((n, n))
    for i in range(n):
        for j in range(i):
            L_w[i, j] = L_w[j, i] = -w[i, j]
    for i in range(n):
        L_w[i, i] = -sum(L_w[i, :])
    e0 = stress(Z, D)
    while True:
        L_Z = np.zeros((n, n))
        for i in range(n):
            Z_i = Z[i, :]
            for j in range(i):
                Z_j = Z[j, :]
                L_Z[i, j] = L_Z[j, i] = -delta[i, j] * inv(np.linalg.norm(Z_i - Z_j))
        for i in range(n):
            L_Z[i][i] = -sum(L_Z[i, :])
        Z[1:, 0] = cg(L_w[1:, 1:], (L_Z @ Z[:, 0])[1:])[0]
        Z[1:, 1] = cg(L_w[1:, 1:], (L_Z @ Z[:, 1])[1:])[0]
        e = stress(Z, D)
        if (e0 - e) / e0 < epsilon:
            break
        e0 = e
    return {u: Z[i] for i, u in enumerate(graph.nodes())}

In [ ]:
import random
import numpy as np
import math

def floyd_warshall_dij(graph):
  print("floyd start")
  idx = 0
  label_dict = {}
  for u in graph.nodes:
    label_dict[u] = idx
    idx += 1

  dij = []
  shortest_path_dict  = nx.floyd_warshall(graph)
  max_d = -1
  for i , dics in shortest_path_dict.items():
    vec = [0] * len(graph.nodes)
    for dic in dics.items():
      vec[label_dict[dic[0]]] = dic[1]
      max_d = max(max_d, dic[1])
    dij.append(vec)
  print("floyd end")
  return dij, max_d

def SGD(graph):
  n = graph.number_of_nodes()
  k = 2
  d, max_d = floyd_warshall_dij(graph)
  print(max_d)

  X = np.ones((n, k))
  size = UNIT_EDGE_LENGTH * graph.number_of_nodes() ** 0.5
  for i in range(n):
    for j in range(k):
      X[i][j] =   random.random() * size

  eta = 1000
  lamda = 0.8
  tmax = 15
  EPS = 0.1

  ordered_each = []

  for i in range(n):
    for j in range(i+1, n):
      ordered_each.append((i, j))

  for t in range(tmax):
    random.shuffle(  ordered_each )
    for i,j in ordered_each:
      wij  = d[i][j] ** -2
      mu = min(eta*wij, 1)
      nodes_norm = np.linalg.norm(X[i]-X[j])
      r = ( nodes_norm  -   d[i][j])  /  2.0  / nodes_norm * (X[i] - X[j])
      X[i] -= mu * r
      X[j] += mu * r
    eta = (max_d **2 ) *  math.exp(-lamda*t)
    eta = max(eta, EPS)
    print(t, eta)
    # print(eta)
    # print(X)
  return  { u : (v[0], v[1])  for (u, v) in zip(graph.nodes , X) }

In [94]:
import random


def MaxMinRandomSP(graph, h):
  graph_nodes = [  i for i in graph.nodes]
  source = random.choice(graph_nodes)

  pivots  = [source]
  for i in range(h-1):
    p = nx.shortest_path_length(graph, source=source)
    max_len = -1
    pivot = -1
    for target in graph.nodes:
      if(p[target] > max_len and not ( target in set(pivots))):
        max_len = p[target]
        pivot = target
    pivots.append(pivot)
    source = pivot
  return pivots

def  SparseShortestPaths(graph, pivots):
  d = {}
  for source in pivots:
    p = nx.shortest_path_length(graph, source=source)

    target_map = {}
    for target in graph.nodes:
      target_map[target] = p[target]
    d[source] = target_map
  print("d: ",d)
  return d

def SSGD(graph,  h):
  n = graph.number_of_nodes()
  k = 2
  pivots = MaxMinRandomSP(graph, h)
  print("finish pivots")
  d  = SparseShortestPaths(graph, pivots)
  print("finishd")
  print(d)

  for p in pivots:
    for i in graph.nodes:
      s = [ j for j in d].length

  for i in graph.edges
    print()

  X = np.ones((n, k))
  size = UNIT_EDGE_LENGTH * graph.number_of_nodes() ** 0.5
  for i in range(n):
    for j in range(k):
      X[i][j] =   random.random() * size

  ordered_each = []
  for i in range(n):
    for j in range(i+1, n):
      ordered_each.append((i, j))


In [95]:
import json
import networkx as nx
name =  '3elt'
graph = nx.node_link_graph(json.load(open(f'graph/{name}.json')))


print(SSGD(graph, 10))
#print(graph)
#print(graph.nodes)
#print(graph.edges)
#pos = SGD(graph)
# print(pos)
# print(graph.edges)

finish pivots
d:  {1627: {2: 21, 1: 22, 4: 22, 5: 21, 3: 20, 7: 20, 6: 20, 9: 19, 8: 23, 10: 21, 15: 22, 11: 20, 12: 20, 14: 19, 13: 19, 18: 24, 23: 23, 16: 18, 17: 20, 26: 21, 19: 19, 22: 19, 29: 20, 20: 18, 21: 18, 34: 22, 24: 17, 30: 20, 31: 21, 183: 24, 327: 25, 25: 18, 27: 19, 28: 17, 33: 17, 39: 18, 46: 19, 272: 23, 37: 16, 32: 18, 50: 22, 64: 22, 35: 19, 38: 20, 36: 17, 148: 21, 282: 20, 41: 21, 43: 21, 69: 22, 40: 18, 48: 16, 57: 17, 2142: 23, 42: 19, 44: 16, 45: 17, 52: 15, 47: 20, 157: 18, 172: 19, 49: 18, 51: 19, 59: 21, 66: 22, 53: 20, 100: 22, 54: 16, 58: 15, 55: 17, 1791: 20, 62: 21, 73: 15, 84: 16, 56: 18, 143: 23, 151: 23, 60: 19, 76: 14, 65: 20, 61: 16, 67: 15, 63: 17, 68: 18, 173: 17, 77: 14, 83: 22, 93: 22, 72: 19, 71: 17, 74: 16, 79: 21, 70: 18, 4561: 23, 80: 20, 128: 23, 81: 15, 75: 19, 216: 23, 78: 18, 82: 17, 87: 19, 107: 14, 110: 15, 85: 16, 86: 19, 103: 13, 91: 14, 88: 18, 97: 20, 101: 21, 94: 19, 90: 15, 89: 17, 126: 21, 200: 16, 95: 16, 96: 18, 98: 18, 99: 18

In [ ]:
# 描画結果の表示
nx.draw(graph, pos)

In [ ]:
# 描画結果の評価
import itertools
import math
d = dict(nx.all_pairs_dijkstra_path_length(graph, weight=lambda u, v, e: 30))
s = 0
for u, v in itertools.combinations(graph.nodes, 2):
    s += (math.hypot(pos[u][0] - pos[v][0], pos[u][1] - pos[v][0]) - d[u][v]) ** 2 / d[u][v] ** 2
s

In [ ]:
# 描画結果の保存
json.dump(pos, open(f'result/{name}.json', 'w'))